# TQQQ MA200 Backtesting Strategy

## Strategy Rules:
- **BUY TQQQ** when: QQQ > MA200 × 1.04 AND QQQ drops >= 1% intraday (Low vs prev Close)
- **SELL** when: QQQ < MA200 × 0.97

This notebook walks through each step of the backtest.

## Step 1: Import Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## Step 2: Define Parameters

Adjust these parameters to customize your backtest:

In [ ]:
# Backtest parameters
START_DATE = "2015-01-01"
END_DATE = "2025-01-17"
INITIAL_CAPITAL = 100000

# Strategy parameters
MA_PERIOD = 200
BUY_THRESHOLD = 1.04      # QQQ > MA200 * 1.04
SELL_THRESHOLD = 0.97     # QQQ < MA200 * 0.97
INTRADAY_DROP_THRESHOLD = -0.01  # QQQ intraday drop >= 1%

print(f"Backtest Period: {START_DATE} to {END_DATE}")
print(f"Initial Capital: ${INITIAL_CAPITAL:,}")
print(f"\nStrategy Rules:")
print(f"  BUY when: QQQ > MA{MA_PERIOD} × {BUY_THRESHOLD} AND intraday drop >= {abs(INTRADAY_DROP_THRESHOLD)*100}%")
print(f"  SELL when: QQQ < MA{MA_PERIOD} × {SELL_THRESHOLD}")

## Step 3: Fetch Historical Data

In [ ]:
# Fetch extra data for MA200 calculation
extended_start = pd.to_datetime(START_DATE) - pd.Timedelta(days=300)

print(f"Fetching QQQ data from {extended_start.date()}...")
qqq_data = yf.download('QQQ', start=extended_start, end=END_DATE, progress=False)

print(f"Fetching TQQQ data from {extended_start.date()}...")
tqqq_data = yf.download('TQQQ', start=extended_start, end=END_DATE, progress=False)

# Handle multi-level columns from yfinance
if isinstance(qqq_data.columns, pd.MultiIndex):
    qqq_data.columns = qqq_data.columns.get_level_values(0)
if isinstance(tqqq_data.columns, pd.MultiIndex):
    tqqq_data.columns = tqqq_data.columns.get_level_values(0)

print(f"\nQQQ data: {qqq_data.shape[0]} rows")
print(f"TQQQ data: {tqqq_data.shape[0]} rows")

In [ ]:
# Preview the data
print("QQQ Data (last 5 rows):")
qqq_data.tail()

In [ ]:
print("TQQQ Data (last 5 rows):")
tqqq_data.tail()

## Step 4: Calculate Technical Indicators

In [ ]:
# Calculate 200-day moving average
qqq_data['MA200'] = qqq_data['Close'].rolling(window=MA_PERIOD).mean()

# Calculate intraday drop: (Low - Previous Close) / Previous Close
qqq_data['Prev_Close'] = qqq_data['Close'].shift(1)
qqq_data['Intraday_Drop'] = (qqq_data['Low'] - qqq_data['Prev_Close']) / qqq_data['Prev_Close']

# Calculate buy/sell threshold levels
qqq_data['Buy_Level'] = qqq_data['MA200'] * BUY_THRESHOLD
qqq_data['Sell_Level'] = qqq_data['MA200'] * SELL_THRESHOLD

print("Indicators calculated!")
qqq_data[['Close', 'Low', 'MA200', 'Intraday_Drop', 'Buy_Level', 'Sell_Level']].tail(10)

## Step 5: Visualize QQQ with MA200

In [ ]:
# Filter to backtest period for visualization
qqq_plot = qqq_data[qqq_data.index >= START_DATE]

plt.figure(figsize=(14, 6))
plt.plot(qqq_plot.index, qqq_plot['Close'], label='QQQ', linewidth=1)
plt.plot(qqq_plot.index, qqq_plot['MA200'], label='MA200', linewidth=1, alpha=0.8)
plt.plot(qqq_plot.index, qqq_plot['Buy_Level'], label=f'Buy Level (MA200×{BUY_THRESHOLD})', 
         linestyle='--', alpha=0.6, color='green')
plt.plot(qqq_plot.index, qqq_plot['Sell_Level'], label=f'Sell Level (MA200×{SELL_THRESHOLD})', 
         linestyle='--', alpha=0.6, color='red')

plt.title('QQQ with MA200 and Trading Thresholds')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.tight_layout()
plt.show()

## Step 6: Generate Trading Signals

In [ ]:
# Filter to backtest period
qqq_data = qqq_data[qqq_data.index >= START_DATE].copy()
tqqq_data = tqqq_data[tqqq_data.index >= START_DATE].copy()

# Align data on common dates
common_dates = qqq_data.index.intersection(tqqq_data.index)
qqq_data = qqq_data.loc[common_dates]
tqqq_data = tqqq_data.loc[common_dates]

print(f"Backtest period: {common_dates[0].date()} to {common_dates[-1].date()}")
print(f"Trading days: {len(common_dates)}")

In [ ]:
# Create signals DataFrame
signals = pd.DataFrame(index=qqq_data.index)
signals['QQQ_Close'] = qqq_data['Close']
signals['QQQ_Low'] = qqq_data['Low']
signals['TQQQ_Close'] = tqqq_data['Close']
signals['MA200'] = qqq_data['MA200']
signals['Intraday_Drop'] = qqq_data['Intraday_Drop']
signals['Buy_Level'] = qqq_data['Buy_Level']
signals['Sell_Level'] = qqq_data['Sell_Level']

# Buy condition: QQQ > MA200 * 1.04 AND intraday drop >= 1%
signals['Buy_Condition'] = (
    (signals['QQQ_Close'] > signals['Buy_Level']) &
    (signals['Intraday_Drop'] <= INTRADAY_DROP_THRESHOLD)
)

# Sell condition: QQQ < MA200 * 0.97
signals['Sell_Condition'] = signals['QQQ_Close'] < signals['Sell_Level']

print("Days meeting BUY condition:", signals['Buy_Condition'].sum())
print("Days meeting SELL condition:", signals['Sell_Condition'].sum())

In [ ]:
# Generate position signals (1 = long TQQQ, 0 = cash)
signals['Signal'] = 0
position = 0

for i in range(len(signals)):
    if position == 0 and signals['Buy_Condition'].iloc[i]:
        position = 1
        signals.iloc[i, signals.columns.get_loc('Signal')] = 1
    elif position == 1 and signals['Sell_Condition'].iloc[i]:
        position = 0
        signals.iloc[i, signals.columns.get_loc('Signal')] = 0
    else:
        signals.iloc[i, signals.columns.get_loc('Signal')] = position

# Track position changes
signals['Position_Change'] = signals['Signal'].diff().fillna(0)

print("Signal generation complete!")

## Step 7: View Trade Log

In [ ]:
# Extract trades
trades = signals[signals['Position_Change'] != 0].copy()
trades['Action'] = trades['Position_Change'].apply(lambda x: 'BUY' if x == 1 else 'SELL')
trades['Intraday_Drop_Pct'] = trades['Intraday_Drop'].apply(lambda x: f"{x*100:.2f}%")

trade_log = trades[['Action', 'QQQ_Close', 'QQQ_Low', 'TQQQ_Close', 'MA200', 'Intraday_Drop_Pct']].copy()
trade_log.columns = ['Action', 'QQQ Close', 'QQQ Low', 'TQQQ Price', 'MA200', 'Intraday Drop']

print(f"Total Trades: {len(trades[trades['Action'] == 'BUY'])} round trips")
print("\nTrade Log:")
trade_log

## Step 8: Run Backtest Simulation

In [ ]:
# Create portfolio DataFrame
portfolio = pd.DataFrame(index=signals.index)

# Calculate TQQQ daily returns
tqqq_returns = tqqq_data['Close'].pct_change().fillna(0)

# Use shifted signal to avoid look-ahead bias
portfolio['Position'] = signals['Signal'].shift(1).fillna(0)
portfolio['TQQQ_Return'] = tqqq_returns
portfolio['Strategy_Return'] = portfolio['Position'] * portfolio['TQQQ_Return']

# Calculate cumulative returns
portfolio['Cumulative_TQQQ'] = (1 + portfolio['TQQQ_Return']).cumprod()
portfolio['Cumulative_Strategy'] = (1 + portfolio['Strategy_Return']).cumprod()

# Calculate portfolio values
portfolio['TQQQ_Value'] = INITIAL_CAPITAL * portfolio['Cumulative_TQQQ']
portfolio['Strategy_Value'] = INITIAL_CAPITAL * portfolio['Cumulative_Strategy']

print("Backtest simulation complete!")
portfolio[['Position', 'TQQQ_Return', 'Strategy_Return', 'Strategy_Value', 'TQQQ_Value']].tail(10)

## Step 9: Calculate Performance Metrics

In [ ]:
def calculate_metrics(portfolio, initial_capital):
    """Calculate performance metrics."""
    strategy_returns = portfolio['Strategy_Return']
    tqqq_returns = portfolio['TQQQ_Return']
    
    # Total return
    total_strategy_return = (portfolio['Strategy_Value'].iloc[-1] / initial_capital - 1) * 100
    total_tqqq_return = (portfolio['TQQQ_Value'].iloc[-1] / initial_capital - 1) * 100
    
    # Annualized return
    trading_days = len(portfolio)
    years = trading_days / 252
    ann_strategy = ((1 + total_strategy_return / 100) ** (1 / years) - 1) * 100
    ann_tqqq = ((1 + total_tqqq_return / 100) ** (1 / years) - 1) * 100
    
    # Volatility (annualized)
    strategy_vol = strategy_returns.std() * np.sqrt(252) * 100
    tqqq_vol = tqqq_returns.std() * np.sqrt(252) * 100
    
    # Sharpe ratio
    sharpe_strategy = ann_strategy / strategy_vol if strategy_vol > 0 else 0
    sharpe_tqqq = ann_tqqq / tqqq_vol if tqqq_vol > 0 else 0
    
    # Maximum drawdown
    strategy_cummax = portfolio['Strategy_Value'].cummax()
    strategy_dd = (portfolio['Strategy_Value'] - strategy_cummax) / strategy_cummax
    max_dd_strategy = strategy_dd.min() * 100
    
    tqqq_cummax = portfolio['TQQQ_Value'].cummax()
    tqqq_dd = (portfolio['TQQQ_Value'] - tqqq_cummax) / tqqq_cummax
    max_dd_tqqq = tqqq_dd.min() * 100
    
    # Time in market
    time_in_market = portfolio['Position'].mean() * 100
    
    return {
        'Total Return (%)': [round(total_strategy_return, 2), round(total_tqqq_return, 2)],
        'Annualized Return (%)': [round(ann_strategy, 2), round(ann_tqqq, 2)],
        'Volatility (%)': [round(strategy_vol, 2), round(tqqq_vol, 2)],
        'Sharpe Ratio': [round(sharpe_strategy, 2), round(sharpe_tqqq, 2)],
        'Max Drawdown (%)': [round(max_dd_strategy, 2), round(max_dd_tqqq, 2)],
        'Final Value ($)': [round(portfolio['Strategy_Value'].iloc[-1], 2), 
                           round(portfolio['TQQQ_Value'].iloc[-1], 2)],
    }, time_in_market

metrics, time_in_market = calculate_metrics(portfolio, INITIAL_CAPITAL)

# Display as DataFrame
metrics_df = pd.DataFrame(metrics, index=['Strategy', 'Buy & Hold TQQQ']).T
print("\n" + "="*50)
print("PERFORMANCE COMPARISON")
print("="*50)
print(metrics_df)
print(f"\nTime in Market: {time_in_market:.2f}%")
print(f"Number of Trades: {len(trades[trades['Action'] == 'BUY'])}")

## Step 10: Visualize Results

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Portfolio value comparison
ax1 = axes[0]
ax1.plot(portfolio.index, portfolio['Strategy_Value'], label='Strategy', linewidth=1.5, color='blue')
ax1.plot(portfolio.index, portfolio['TQQQ_Value'], label='Buy & Hold TQQQ', linewidth=1.5, color='orange', alpha=0.7)
ax1.set_title('Portfolio Value: Strategy vs Buy & Hold TQQQ', fontsize=12)
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Plot 2: QQQ with signals
ax2 = axes[1]
ax2.plot(signals.index, signals['QQQ_Close'], label='QQQ', linewidth=1, color='black')
ax2.plot(signals.index, signals['MA200'], label='MA200', linewidth=1, color='blue', alpha=0.7)
ax2.plot(signals.index, signals['Buy_Level'], label=f'Buy Level (MA200×{BUY_THRESHOLD})', 
         linewidth=1, linestyle='--', color='green', alpha=0.5)
ax2.plot(signals.index, signals['Sell_Level'], label=f'Sell Level (MA200×{SELL_THRESHOLD})', 
         linewidth=1, linestyle='--', color='red', alpha=0.5)

# Mark buy/sell points
buys = signals[signals['Position_Change'] == 1]
sells = signals[signals['Position_Change'] == -1]
ax2.scatter(buys.index, buys['QQQ_Close'], marker='^', color='green', s=100, label='Buy', zorder=5)
ax2.scatter(sells.index, sells['QQQ_Close'], marker='v', color='red', s=100, label='Sell', zorder=5)

ax2.set_title('QQQ with MA200 and Trading Signals', fontsize=12)
ax2.set_ylabel('QQQ Price ($)')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# Plot 3: Position over time
ax3 = axes[2]
ax3.fill_between(portfolio.index, portfolio['Position'], step='post', alpha=0.5, color='blue')
ax3.set_title('Position Over Time (1 = Long TQQQ, 0 = Cash)', fontsize=12)
ax3.set_ylabel('Position')
ax3.set_xlabel('Date')
ax3.set_ylim(-0.1, 1.1)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 11: Drawdown Analysis

In [ ]:
# Calculate drawdowns
portfolio['Strategy_Drawdown'] = (portfolio['Strategy_Value'] / portfolio['Strategy_Value'].cummax() - 1) * 100
portfolio['TQQQ_Drawdown'] = (portfolio['TQQQ_Value'] / portfolio['TQQQ_Value'].cummax() - 1) * 100

plt.figure(figsize=(14, 5))
plt.fill_between(portfolio.index, portfolio['Strategy_Drawdown'], 0, alpha=0.5, label='Strategy', color='blue')
plt.fill_between(portfolio.index, portfolio['TQQQ_Drawdown'], 0, alpha=0.5, label='Buy & Hold TQQQ', color='orange')
plt.title('Drawdown Comparison')
plt.ylabel('Drawdown (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Strategy Max Drawdown: {portfolio['Strategy_Drawdown'].min():.2f}%")
print(f"Buy & Hold Max Drawdown: {portfolio['TQQQ_Drawdown'].min():.2f}%")

## Step 12: Export Results (Optional)

In [ ]:
# Uncomment to export results

# Export trade log
# trade_log.to_csv('trade_log.csv')
# print("Trade log exported to trade_log.csv")

# Export portfolio data
# portfolio.to_csv('portfolio_history.csv')
# print("Portfolio history exported to portfolio_history.csv")

# Export signals
# signals.to_csv('signals.csv')
# print("Signals exported to signals.csv")

print("Uncomment the lines above to export data to CSV files.")

---
## Experiment Ideas

Try modifying the parameters in **Step 2** to see how they affect performance:

1. **Change MA period**: Try MA100 or MA50 instead of MA200
2. **Adjust thresholds**: Try BUY_THRESHOLD = 1.05 or SELL_THRESHOLD = 0.95
3. **Change daily loss requirement**: Try -0.02 (2% loss) or -0.005 (0.5% loss)
4. **Different time periods**: Focus on specific market conditions (bull/bear markets)
5. **Add transaction costs**: Subtract 0.1% per trade from returns